# import files

In [ ]:
pip install sentence_transformers

     |████████████████████████████████| 81kB 5.6MB/s 
     |████████████████████████████████| 2.5MB 11.3MB/s 
     |████████████████████████████████| 1.2MB 36.7MB/s 
     |████████████████████████████████| 3.3MB 50.6MB/s 
     |████████████████████████████████| 901kB 42.0MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.2.0-cp37-none-any.whl size=123339 sha256=7c8ade2daddf39af06f508d27d0b251d097814392f0c0e8213983fa44d585cbb
  Stored in directory: /root/.cache/pip/wheels/0f/06/f7/faaa96fdda87462b4fd5c47b343340e9d5531ef70d0eef8242
Successfully built sentence-transformers


# A simple Semantic search 
https://www.sbert.net/examples/applications/semantic-search/README.html

Semantic search seeks to improve search accuracy by understanding the content of the search query. In contrast to traditional search engines, that only finds documents based on lexical matches, semantic search can also find synonyms.

Also available:
Types of Search- Symmetric and Asymmetric
Code for Speed optimization of encoding through Cuda

In [ ]:
x_train = ['where can i get a massage oil']
 
xx_train = ['Yes. It is right behind Kahrama in the National area.', 'whats the name of the shop?', 
            "It's called Naseem Al-Nadir. Right next to the Smartlink shop. You'll find the chinese salesgirls at affordable prices there.", 
            'dont want girls;want oil', "Try Both ;) I'am just trying to be helpful. On a serious note - Please go there. you'll find what you are looking for."]

In [ ]:
 from sentence_transformers import SentenceTransformer, util
import torch
 
embedder = SentenceTransformer('paraphrase-distilroberta-base-v1')
 
# Corpus with example sentences
corpus = xx_train
 
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
 
# Query sentences:
queries = x_train
 
 
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
 
    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
 
    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")
 
    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: {:.4f})".format(score))
 
    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """






Query: where can i get a massage oil

Top 5 most similar sentences in corpus:
whats the name of the shop? (Score: 0.1681)
dont want girls;want oil (Score: 0.1193)
It's called Naseem Al-Nadir. Right next to the Smartlink shop. You'll find the chinese salesgirls at affordable prices there. (Score: 0.1135)
Yes. It is right behind Kahrama in the National area. (Score: 0.0972)
Try Both ;) I'am just trying to be helpful. On a serious note - Please go there. you'll find what you are looking for. (Score: 0.0864)


# A simple semantic search function of whole data.
Custom simple short Cosine function added in cells too.

In [ ]:
# semantic search with full sets

from sentence_transformers import SentenceTransformer, util
import torch
embedder = SentenceTransformer('paraphrase-distilroberta-base-v1')

# read data from file
data = pd.read_csv(filepath)
data.describe()

# #preparing data for models 
# queries = list(data['question'][::100]) #since 100 copies of it were made for one pair, won't be necessary until Sentence Pair is needed
# passages = list(data['answer']) 
# data = data.drop_duplicates(subset= ['answer'], keep = 'last') #multiple copies of answer gave skewed repetitive results therefore removing duplicate answers

# #We limit the number of passages so that the encoding is faster.
# #Just uncomment the next line to encode the complete corpus of ~500+ paragraphs
# print("Passages:", len(passages))
# print("Queries:", len(queries))
# print("Data Now:", len(data))
print(data.describe())

# ORRRRRRRRRR

queries = list(data['question'].unique())
features = list(data['feature'].unique())
passages = list(data['answer'].unique())

print(len(queries), len(features), len(passages))


Passages: 19990
Queries: 200
Data Now: 13854
                                                  feature  ... labels
count                                               13854  ...  13854
unique                                                200  ...      3
top     Where can I get International Driving License ...  ...    Bad
freq                                                  100  ...  11324

[4 rows x 4 columns]


In [ ]:
# Corpus with example sentences
corpus = passages
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [ ]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
 
    # We use cosine-similarity and torch.topk to find the highest 5 scores

    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits[:5]:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))


Shav what kind of massage does she do? is it filipino massage? i missed that a lot!!! (Score: 0.5462)
pls let me know where is that massage done? i want to try it.thanks (Score: 0.5412)
"You can get tea tree oil at the Body Shop (or you could) and I've seen Eucalyptus oil at FFC in the ""medicine"" section. Have you tried Occitane? I don't know what they carry but might be worth a shot if you're in the vicinity anyways (have a shop at Royal Plaza and I think Landmark?) Signature line > ""You can't fix stupid""" (Score: 0.5145)
I have never been for a massage! Can you believe it? Maybe I should go and try too! :) (Score: 0.5091)
Most massages in Qatar are a waste of money. All they do is just rub some oil. No body does deep tissue massage here. (Score: 0.5024)
i brouhgt my wife and my child on visit visa;first i stamped their visa for 6 month then last month i extended for one month so total 7 month is over; can it be extended; if yes then how and which branch should i go (Score: 0.7107

In [ ]:
# #Compute cosine-similarities for each sentence in PASSAGE with each other sentence in PASSAGE

# #Compute cosine-similarities for each sentence with each other sentence
# cosine_scores = util.pytorch_cos_sim(corpus_embeddings, corpus_embeddings)

# #Find the pairs with the highest cosine similarity scores
# pairs = []
# for i in range(len(cosine_scores)-1):
#     for j in range(i+1, len(cosine_scores)):
#         pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

# #Sort scores in decreasing order
# pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

# for pair in pairs[0:10]:
#     i, j = pair['index']
#     print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], pair['score']))

In [ ]:
# #Compute cosine-similarities for each sentence in QUERY with each other sentence in PASSAGE

# cosine_scores = util.pytorch_cos_sim(query_embeddings, corpus_embeddings)

# #Find the pairs with the highest cosine similarity scores
# pairs = []
# for i in range(len(cosine_scores)-1):
#     for j in range(i+1, len(cosine_scores)):
#         pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

# #Sort scores in decreasing order
# pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

# for pair in pairs[0:10]:
#     i, j = pair['index']
#     print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], pair['score']))

# Retrieve and Re-rank Model
https://www.sbert.net/examples/applications/retrieve_rerank/README.html

Used BiEncoder to Retrieve
BM25 first ranking, then CrossEncoder to Re-rank


DATA: Subtask C training Data
Queries are Questions, Passages are all Answers.


In [ ]:
filepath = '/content/TRAIN-BASE.csv' #file locally read


In [ ]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import time
import gzip
import os
import torch
import pandas as pd


#We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
model_name = 'paraphrase-distilroberta-base-v1'
bi_encoder = SentenceTransformer(model_name)
top_k = 100     #Number of passages we want to retrieve with the bi-encoder

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')


In [ ]:
# read data from file
data = pd.read_csv(filepath)
data.describe()

,feature,question,answer,labels
count,19990,19990,19990,19990
unique,200,200,13854,3
top,Bottled water prices in Qatar,Dear all; May i know pls; for the resident vis...,IMAGE_LINK,Bad
freq,100,100,20,16319


In [ ]:
#preparing data for models 


# # passages = [] #has responses that we will predict.
# # queries = [] #has questions/queries we will search responses for.


# queries = list(data['question'][::100]) #since 100 copies of it were made for one pair, won't be necessary until Sentence Pair is needed
# passages = list(data['answer']) 

# data = data.drop_duplicates(subset= ['answer'], keep = 'last') #multiple copies of answer gave skewed repetitive results therefore removing duplicate answers

# #We limit the number of passages so that the encoding is faster.
# #Just uncomment the next line to encode the complete corpus of ~500+ paragraphs
# print("Passages:", len(passages))
# print("Queries:", len(queries))
# print("Data Now:", len(data))
data.describe()


# ORRRRRRRRRR

queries = list(data['question'].unique())
features = list(data['feature'].unique())
passages = list(data['answer'].unique())

print(len(queries), len(features), len(passages))

Passages: 19990
Queries: 200
Data Now: 13854


,feature,question,answer,labels
count,13854,13854,13854,13854
unique,200,200,13854,3
top,Massage oil,Where I can buy good oil for massage?,he is from Pakistan. are there any possibiliti...,Bad
freq,100,100,1,11324


In [ ]:
queries[:10]

['Where I can buy good oil for massage?',
 "Dear Members; I have my wife in qatar on family visit visa; now it's gonna over 6 months; can i get more 2/3 months visa?? what is the procedure?? pls inform.",
 'want to extend the visit visa. What are the procedures and how long i get the extension. thanks for the answers...',
 'How can my boyfriend who currently works for a government office there get me and our baby a visit visa? How long can we stay there in doha and can we also get married there as well? Can we stay together in his house there? Can you give us advice and the steps to take for getting visa and requirements for marriage there in Qatar? It will be greatly appreciated.Thanks in advance',
 'I begin employment in Qatar Aug 09 and have just been quoted 1000 to have them flown over. Is it worth the hassle of customs or should i buy new? How do prices compare UK Next with Qatar Next?',
 'My employer have recruited me as a Business Development Executive in a Labor Visa.I am put o

In [ ]:
passages[:10]

['Yes. It is right behind Kahrama in the National area.',
 'whats the name of the shop?',
 "It's called Naseem Al-Nadir. Right next to the Smartlink shop. You'll find the chinese salesgirls at affordable prices there.",
 'dont want girls;want oil',
 "Try Both ;) I'am just trying to be helpful. On a serious note - Please go there. you'll find what you are looking for.",
 'you mean oil and filter both',
 "Yes Lawa...you couldn't be more right LOL",
 'What they offer?',
 'FU did u try with that salesgirl ?',
 "Swine - No I don't try with salesgirls. My taste is classy ;)"]

In [ ]:
!pip install -U sentence-transformers rank_bm25

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.7/dist-packages (1.2.0)
Requirement already up-to-date: rank_bm25 in /usr/local/lib/python3.7/dist-packages (0.2.1)


In [ ]:
# if model_name == 'msmarco-distilbert-base-v2':
#   embeddings_filepath = 'simplewiki-2020-11-01-msmarco-distilbert-base-v2.pt'
#   if not os.path.exists(embeddings_filepath):
#       util.http_get('http://sbert.net/datasets/simplewiki-2020-11-01-msmarco-distilbert-base-v2.pt', embeddings_filepath)

#   corpus_embeddings = torch.load(embeddings_filepath)
#   corpus_embeddings = corpus_embeddings.float() #Convert embedding file to float
#   if torch.cuda.is_available():
#     corpus_embeddings = corpus_embeddings.to('cuda')
# else:   #Here, we compute the corpus_embeddings from scratch (which can take a while depending on the GPU)


corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)
if torch.cuda.is_available():
  corpus_embeddings = corpus_embeddings.to('cuda')

In [ ]:
# We also compare the results to lexical search (keyword search). Here, we use 
# the BM25 algorithm which is implemented in the rank_bm25 package.

from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np

# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
  tokenized_doc = []
  for token in text.lower().split():
    token = token.strip(string.punctuation)

    if len(token) > 0 and token not in stop_words.ENGLISH_STOP_WORDS:
      tokenized_doc.append(token)
  return tokenized_doc

tokenized_corpus = []
for passage in tqdm(passages):
  tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
#This function will search all wikipedia articles for passages that
#answer the query

# from sentence_transformers import SentenceTransformer, CrossEncoder, util

def search(query):
  print("Input question:", query)

  #BM25 search (lexical search)
  bm25_scores = bm25.get_scores(bm25_tokenizer(query))
  top_n = np.argpartition(bm25_scores, -5)[-5:]
  bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
  bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
  print("Top-5 lexical search (BM25) hits")
  for hit in bm25_hits[0:10]:
      print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))

  ##### Sematic Search #####
  #Encode the query using the bi-encoder and find potentially relevant passages
  question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
  question_embedding = question_embedding.cuda()
  hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
  hits = hits[0]  # Get the hits for the first query

  ##### Re-Ranking #####
  #Now, score all retrieved passages with the cross_encoder
  cross_inp = [[query, passages[hit['corpus_id']]] for hit in hits]
  cross_scores = cross_encoder.predict(cross_inp)

  #Sort results by the cross-encoder scores
  for idx in range(len(cross_scores)):
      hits[idx]['cross-score'] = cross_scores[idx]


  #Output of top-5 hitt
  print("Top-5 Bi-Encoder Retrieval hits")
  hits = sorted(hits, key=lambda x: x['score'], reverse=True)
  for hit in hits[:10]:
      print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))

  print("Top-5 Cross-Encoder Re-ranker hits")
  hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
  for hit in hits[0:10]:
      print("\t{:.3f}\t{}".format(hit['cross-score'], passages[hit['corpus_id']].replace("\n", " ")))

In [ ]:
search(query = "how to get visa in qatar and then jobs?")

Input question: how to get visa in qatar and then jobs?
Top-5 lexical search (BM25) hits
	11.839	Which recruitment consultancy deals with IT Jobs in Qatar for Qatar jobs. plz do advice me ...itz very urgent...
	11.596	I am an Indian female here in Qatar on visit visa. I am in need of a job (Secretarial or any administrative jobs). Can I get a company who can give me visa. Thanks. Contact No. 6654580
	11.596	I am an Indian female here in Qatar on visit visa. I am in need of a job (Secretarial or any administrative jobs). Can I get a company who can give me visa. Thanks. Contact No. 6654580
	11.596	I am an Indian female here in Qatar on visit visa. I am in need of a job (Secretarial or any administrative jobs). Can I get a company who can give me visa. Thanks. Contact No. 6654580
	11.596	I am an Indian female here in Qatar on visit visa. I am in need of a job (Secretarial or any administrative jobs). Can I get a company who can give me visa. Thanks. Contact No. 6654580
Top-5 Bi-Encoder R

In [ ]:
search(query = "WHERE CAN I GET A MANICURE")

Input question: WHERE CAN I GET A MANICURE
Top-5 lexical search (BM25) hits
	7.729	You can get a manicure and pedicure for 100QR at Pinkies. This is the cheapest place I have been able to find since moving here. However; it is not near Villiagio. It is located at The Centre by Mega-mart.
	6.283	Hope12 if u hav experiene in beauty parlours like Henna designing; pedicure manicure; bleacheing etc. etc all tat lady beauty stuff u can do it privately; from ur home; i guess...!!
	6.283	Hope12 if u hav experiene in beauty parlours like Henna designing; pedicure manicure; bleacheing etc. etc all tat lady beauty stuff u can do it privately; from ur home; i guess...!!
	5.872	last time i went there ... marie clare said she going back to french for summer hols. The salon still open for normal stuffs like colouring and manicure pedicure. there is an english hairdresser in riviera garden; but i dont have the contact number.
	5.872	last time i went there ... marie clare said she going back to french 

In [ ]:
search(query = "hot girls?")

Input question: hot girls?
Top-5 lexical search (BM25) hits
	10.122	no worries me will also send you one hot hot friendsheep request :P
	8.566	if you like girls from asia
	8.432	guess i still have some hot QL chicks [img_assist|nid=21285|title=.|desc=|link=none|align=left|width=180|height=180]
	8.234	Girls with short hair cuts .
	8.129	Too hot will be miserable. Don't go to inland sea until October
Top-5 Bi-Encoder Retrieval hits
	0.445	HOW?
	0.419	PP?
	0.416	???
	0.411	hey; i know ur romanian; arent u?lets see? do u date skinny girls like me?
	0.399	is?
	0.398	cool..
	0.389	sweet
	0.377	huwat?! 10k now?
	0.367	polydan?
	0.360	You drunk?
Top-5 Cross-Encoder Re-ranker hits
	-0.580	"If you want great music; lots of dancing;lots of food; and many many many single beautiful women then i recommend going to any hotels ""wedding halls"" on weekends to a traditional qatari wedding! Party usually goes on from 8pm till after midnight! I usually dont make it past 8:05pm given all the high heel sh

In [ ]:
search( query = 'blind date')

Input question: blind date
Top-5 lexical search (BM25) hits
	12.340	Then he might be blind?
	12.340	Then he might be blind?
	10.119	Lols LP ! u mean color blind ?
	10.119	Lols LP ! u mean color blind ?
	9.226	Lolz! What do you expect? A blind date with a decent and good looking girl? :P Come out of dreams dude; be clear. If you are looking for the ladies who work for money or like some Hala Cards; i am sure your inbox would be full by now. Any decent girl; would laugh at your request. :D Best of Luck; though :)
Top-5 Bi-Encoder Retrieval hits
	0.436	First date? Appear to be really shy... wear nothing.
	0.362	double it
	0.335	faith
	0.330	69
	0.327	sweet
	0.326	21 days per year
	0.325	straight arrow; Marriage is not a Test hence why cheat!
	0.306	MM.
	0.304	DP
	0.304	yes 2 days
Top-5 Cross-Encoder Re-ranker hits
	-5.172	Then he might be blind?
	-5.172	Then he might be blind?
	-7.393	I'm colour blind; so i wait for the blinking light at the top of the tree and just stop..
	-7.799	First d

In [ ]:
search( query = 'what places can i visit in qatar as a tourist and how can i get currency exchanged. i dont know arabic and i have us dollars and indian inr')

Input question: what places can i visit in qatar as a tourist and how can i get currency exchanged. i dont know arabic and i have us dollars and indian inr
Top-5 lexical search (BM25) hits
	14.767	Which all places to visit in Qatar? a lot.. as in a lot.
	13.169	No need to worry. Bring whatever currency you have with you. There are hundreds of money exchanges here in Qatar; you can change your money. Even on the airport; there is currency exchange where you can change some of the money; but I know their rates are little lower than the normal market; so better to change your more money in the other exchanges here in Doha.
	13.020	is $400 (US dollars) as per agreement b/w the Qatar and Phil. Government.
	12.041	If you get one offer to work in Qatar; you need to exit from Qatar or pay some amount in Imigration for changing the visit visa to work visit. If y you are in tourist visa; you need not submit NOC.
	12.041	If you get one offer to work in Qatar; you need to exit from Qatar or pay so

# CLASSIFICATION TASK FOR CQA